<img src="NotebookAddons/blackboard-banner.jpg" width="100%" />
<font face="Calibri">
<br>
<font size="5"><b>Exploring SAR Data and SAR Time Series Analysis using Jupyter Notebooks</b></font>

<br>
<font size="4"><b> Franz J Meyer; University of Alaska Fairbanks & Josef Kellndorfer, <a href="http://earthbigdata.com/" target="_blank">Earth Big Data, LLC</a> </b> <br>
<img src="NotebookAddons/UAFLogo_A_647.png" width="170" align="right" />
</font>

<font size="3"> This notebook will introduce you to the analysis of deep multi-temporal SAR image data stacks in the framework of *Jupyter Notebooks*. The Jupyter Notebook environment is easy to launch in any web browser for interactive data exploration with provided or new training data. Notebooks are comprised of text written in a combination of executable python code and markdown formatting including latex style mathematical equations. Another advantage of Jupyter Notebooks is that they can easily be expanded, changed, and shared with new data sets or newly available time series steps. Therefore, they provide an excellent basis for collaborative and repeatable data analysis. <br>

<b>We introduce the following data analysis concepts:</b>

- How to load your own SAR data into Jupyter Notebooks and create a time series stack 
- How to apply calibration constants to covert initial digital number (DN) data into calibrated radar cross section information.
- How to subset images and create a time series of your subset data.
- How to explore the time-series information in SAR data stacks for environmental analysis.

</font>

<hr>
<font face="Calibri" size="5" color="red"> <b>Important Note about JupyterHub</b> </font>
<br><br>
<font face="Calibri" size="3"> <b>Your JupyterHub server will automatically shutdown when left idle for more than 1 hour. Your notebooks will not be lost but you will have to restart their kernels and re-run them from the beginning. You will not be able to seamlessly continue running a partially run notebook.</b> </font>


<hr>
<font face="Calibri">

<font size="5"> <b> 0. Importing Relevant Python Packages </b> </font>

<font size="3">In this notebook we will use the following scientific libraries:
<ol type="1">
    <li> <b><a href="https://pandas.pydata.org/" target="_blank">Pandas</a></b> is a Python library that provides high-level data structures and a vast variety of tools for analysis. The great feature of this package is the ability to translate rather complex operations with data into one or two commands. Pandas contains many built-in methods for filtering and combining data, as well as the time-series functionality. </li>
    <li> <b><a href="https://www.gdal.org/" target="_blank">GDAL</a></b> is a software library for reading and writing raster and vector geospatial data formats. It includes a collection of programs tailored for geospatial data processing. Most modern GIS systems (such as ArcGIS or QGIS) use GDAL in the background.</li>
    <li> <b><a href="http://www.numpy.org/" target="_blank">NumPy</a></b> is one of the principal packages for scientific applications of Python. It is intended for processing large multidimensional arrays and matrices, and an extensive collection of high-level mathematical functions and implemented methods makes it possible to perform various operations with these objects. </li>
    <li> <b><a href="https://matplotlib.org/index.html" target="_blank">Matplotlib</a></b> is a low-level library for creating two-dimensional diagrams and graphs. With its help, you can build diverse charts, from histograms and scatterplots to non-Cartesian coordinates graphs. Moreover, many popular plotting libraries are designed to work in conjunction with matplotlib. </li>
    <li> The <b><a href="https://www.pydoc.io/pypi/asf-hyp3-1.1.1/index.html" target="_blank">asf-hyp3 API</a></b> provides useful functions and scripts for accessing and processing SAR data via the Alaska Satellite Facility's Hybrid Pluggable Processing Pipeline, or HyP3 (pronounced "hype"). </li>
<li><b><a href="https://www.scipy.org/about.html" target="_blank">SciPY</a></b> is a library that provides functions for numerical integration, interpolation, optimization, linear algebra and statistics. </li>

</font>

<font face="Calibri" size="3"> Our first step is to <b>import them:</b> </font>

In [ ]:
!pip install pyproj # this can be removed when pyproj is installed on jupyterHub

In [ ]:
import os # for chdir, getcwd, path.exists
import glob # for glob
import re # for match
import json # for loads
import math # for ceil
import datetime # for date
from pyproj import Proj, transform

import pandas as pd # for DatetimeIndex
import gdal # for gdalbuildvr, gdalmerge, gdalwarp, gdal_translate, Open
from osgeo import gdal # for Info
import numpy as np # for copy, isnan, log10, ma.masked_where, max, mean, min, percentile, power, unique, var, where 
import matplotlib.pylab as plb # for figure, grid, rcParams, savefig
import matplotlib.pyplot as plt
from matplotlib import animation
import matplotlib.patches as patches  # for Rectangle
from matplotlib import rc
from asf_hyp3 import API # for get_products, get_subscriptions, login
import scipy # for signal.savgol_filter()
import scipy.signal

from bokeh.io import output_notebook

from IPython.display import HTML
from IPython.display import Markdown
from IPython.display import display

from asf_notebook import path_exists

from asf_notebook import new_directory
from asf_notebook import earthdata_hyp3_login
from asf_notebook import remove_nan_filled_tifs
from asf_notebook import get_RTC_polarizations
from asf_notebook import remote_jupyter_proxy_url
from asf_notebook import get_power_set
from asf_notebook import AOI
from asf_notebook import handle_old_data
from asf_notebook import input_path
from asf_notebook import get_hyp3_subscriptions
from asf_notebook import gui_date_picker
from asf_notebook import get_slider_vals
from asf_notebook import get_subscription_products_info
from asf_notebook import get_products_dates
from asf_notebook import get_product_info
from asf_notebook import select_parameter
from asf_notebook import select_mult_parameters
from asf_notebook import get_wget_cmd
from asf_notebook import asf_unzip

<font face="Calibri" size="3"><b>Setup matplotlib plotting</b> inside the notebook:</font>

In [ ]:
%matplotlib inline 

<font face="Calibri" size="3"><b>Setup interactive Bokeh plotting</b> inside the notebook:</font>

In [ ]:
output_notebook()

<hr>
<font face="Calibri">

<font size="5"> <b> 1. Load Your Own Data Stack Into the Notebook </b> </font> 

<font size="3"> This notebook assumes that you've created your own data stack over your personal area of interest using the <a href="https://www.asf.alaska.edu/" target="_blank">Alaska Satellite Facility's</a> value-added product system <a href="http://hyp3.asf.alaska.edu/" target="_blank">HyP3</a>. HyP3 is an environment that is used by ASF to prototype value added products and provide them to users to collect feedback. 

This notebook expects <a href="https://media.asf.alaska.edu/uploads/RTC/rtc_atbd_v1.2_final.pdf" target="_blank">Radiometric Terrain Corrected</a> (RTC) image products as input, so be sure to select an RTC process when creating the subscription for your input data within HyP. Prefer a **unique orbit geometry** (ascending or descending) to keep geometric differences between images low. 

We will retrieve HyP3 data via the HyP3 API. As both HyP3 and the Notebook environment sit in the <a href="https://aws.amazon.com/" target="_blank">Amazon Web Services (AWS)</a> cloud, data transfer is quick and cost effective.</font> 
</font>

<hr>
<font face="Calibri" size="3"> To download data from ASF, you need to provide your <a href="https://www.asf.alaska.edu/get-data/get-started/free-earthdata-account/" target="_blank">NASA Earth Data</a> username to the system. Setup an EarthData account if you do not yet have one. <font color='rgba(200,0,0,0.2)'><b>Note that EarthData's End User License Agreement (EULA) applies when accessing the Hyp3 API from this notebook. If you have not acknowleged the EULA in EarthData, you will need to navigate to <a href="https://earthdata.nasa.gov/" target="_blank">EarthData's home page</a> and complete that process.</b></font>
<br><br>
<b>Login to Earthdata:</b> </font> 

In [ ]:
api = earthdata_hyp3_login()

<hr>
<font face="Calibri" size="3"> Before we download anything, create a working directory for this analysis and change into it. 
<br><br>
<b>Select or create a working directory for the analysis:</b></font>

In [ ]:
while True:
    sub_dir = input_path(
        f"\nPlease enter the name of a directory in which to store your data for this analysis.")
    if os.path.exists(sub_dir):
        contents = glob.glob(f'{sub_dir}/*')
        if len(contents) > 0:
            choice = handle_old_data(sub_dir, contents)
            if choice == 1:
                shutil.rmtree(sub_dir)
                os.mkdir(sub_dir)
                break
            if choice == 2:
                break
            else:
                clear_output()
                continue
        else:
            break
    else:
        os.mkdir(sub_dir)
        break

<font face="Calibri" size="3"><b>Change into the analysis directory:</b></font>

In [ ]:
analysis_directory = f"{os.getcwd()}/{sub_dir}"
os.chdir(analysis_directory)
print(f"Current working directory: {os.getcwd()}")

<font face="Calibri" size="3"><b>Create a folder in which to download your RTC products.</b> </font>

In [ ]:
rtc_path = "rtc_products"
new_directory(rtc_path)
products_path = f"{analysis_directory}/{rtc_path}"

<font face="Calibri" size="3"><b>List subscriptions and select one:</b> </font>

In [ ]:
subscriptions = get_hyp3_subscriptions(api)
if len(subscriptions) > 1:
    display(Markdown("<text style='color:red;'>Note: After selecting a subscription, you must select the next cell before hitting the 'Run' button or typing Shift/Enter.</text>"))
    display(Markdown("<text style='color:red;'>Otherwise, you will simply rerun this code cell.</text>"))
print('\nSelect a Subscription:')
subscription_id = select_parameter('', subscriptions)
subscription_id

<font face="Calibri" size="3"><b>Save the selected subscription ID:</b> </font>

In [ ]:
subscription_id = subscription_id.value.split(':')[0]
print(subscription_id)

<font face="Calibri" size="3"><b>Select a date range of products to download:</b> </font>

In [ ]:
display(Markdown("<text style='color:red;'>Note: After selecting a date range, you should select the next cell before hitting the 'Run' button or typing Shift/Enter.</text>"))
display(Markdown("<text style='color:red;'>Otherwise, you may simply rerun this code cell.</text>"))
print('\nSelect a Date Range:')
products_info = get_subscription_products_info(subscription_id, api)
dates = get_products_dates(products_info)
date_picker = gui_date_picker(dates)
date_picker

<font face="Calibri" size="3"><b>Save the selected date range:</b> </font>

In [ ]:
date_range = get_slider_vals(date_picker)
date_range[0] = date_range[0].date()
date_range[1] = date_range[1].date()
print(f"Date Range: {str(date_range[0])} to {str(date_range[1])}")

<font face="Calibri" size="3"><b>Gather the available paths, flight directions, and download_urls for the subscription, inside the selected date range:</b></font>

In [ ]:
display(Markdown("<text style='color:red;'><text style='font-size:150%;'>This may take some time for large subscriptions...</text></text>"))
product_info = get_product_info(products_info, date_range)
display(Markdown(f"<text style=color:blue><text style='font-size:175%;'>Done.</text></text>"))
paths = set(product_info['paths'])
paths.add('All Paths')

<font face="Calibri" size="3"><b>Select a path or paths (use shift or ctrl to select multiple paths):</b></font>

In [ ]:
display(Markdown("<text style='color:red;'>Note: After selecting a path, you must select the next cell before hitting the 'Run' button or typing Shift/Enter.</text>"))
display(Markdown("<text style='color:red;'>Otherwise, you will simply rerun this code cell.</text>"))
print('\nSelect a Path:')
path_choice = select_mult_parameters('', paths)
path_choice

<font face="Calibri" size="3"><b>Save the selected flight path/s:</b></font>

In [ ]:
fp = path_choice.value
if fp:
    if 'All Paths' in fp:
        flight_path = None
    else:
        flight_path = ""
        for pth in fp:
            if flight_path == "":
                flight_path = f"{pth}"
            else:
                flight_path = f"{flight_path}, {pth}"
    if flight_path:
        print(f"Flight Path: {flight_path}")
    else:
        print('Flight Path: All Paths')
else:
    print("WARNING: You must select a flight path in the previous cell, then rerun this cell.")

<font face="Calibri" size="3"><b>Select an orbit Direction:</b></font>

In [ ]:
valid_directions = set()
for i, path in enumerate(product_info['paths']):
    if not flight_path or path in flight_path:
        valid_directions.add(product_info['directions'][i])
if len(valid_directions) > 1:
    display(Markdown("<text style='color:red;'>Note: After selecting a flight direction, you must select the next cell before hitting the 'Run' button or typing Shift/Enter.</text>"))
    display(Markdown("<text style='color:red;'>Otherwise, you will simply rerun this code cell.</text>"))
print('\nSelect a Flight Direction:')
direction_choice = select_parameter('Direction:', valid_directions)
direction_choice

<font face="Calibri" size="3"><b>Save the selected orbit direction:</b></font>

In [ ]:
direction = direction_choice.value
print(f"Orbit Direction: {direction}")

<font face="Calibri" size="3"><b>Create a list of download_urls within the date range, filtered by orbit direction and flight path:</b> </font>

In [ ]:
download_urls = []
for i, orbit_dir in enumerate(product_info['directions']):
    if orbit_dir == direction:
        if flight_path == None or product_info['paths'][i] == str(flight_path):  #change to allow mult flight paths
            download_urls.append(product_info['urls'][i])
download_urls.sort()
print(f"There are {len(download_urls)} products to download.")

<font face="Calibri" size="3"><b>Download the products, unzip them into the rtc_products directory, and delete the zip files:</b> </font>

In [ ]:
if path_exists(products_path):
    product_count = 1
    print(f"\nSubscription ID: {subscription_id}")
    for url in download_urls:
        print(f"\nProduct Number {product_count} of {len(download_urls)}:")
        product_count += 1
        product = url.split('/')[5]
        filename = f"{products_path}/{product}"
        # if not already present, we need to download and unzip products
        if not os.path.exists(filename.split('.zip')[0]):
            print(
                f"\n{product} is not present.\nDownloading from {url}")
            cmd = get_wget_cmd(url)
            !$cmd
            print(f"\n")
            asf_unzip(products_path, product)
            print(f"product: {product}")
            try:
                os.remove(product)
            except OSError:
                pass
            print(f"\nDone.")
        else:
            print(f"{filename} already exists.")
    display(Markdown(f"<text style=color:blue><text style='font-size:150%;'>ALL PRODUCTS DOWNLOADED</text></text>"))

<hr>
<font face="Calibri" size="3"><b>Determine the subscription's process type</b>, which we need in order to determine the file paths to the tiffs.</font>

In [ ]:
subscription_info = api.get_subscription(subscription_id)
process_type = subscription_info['process_id']

<font face="Calibri" size="3"><b>Create a variable called <i>paths</i>, that holds the paths to the images</b>, which varies based on process type:</font>

In [ ]:
polarizations = get_RTC_polarizations(process_type, rtc_path)
polarization_power_set = get_power_set(polarizations, 2)

In [ ]:
polarization_choice = select_parameter('Polarizations:', sorted(polarization_power_set))
polarization_choice

In [ ]:
polarization = polarization_choice.value
if len(polarization) == 2:
    paths = f"{rtc_path}/*/*{polarization}.tif"
    dbl_polar = False
else:
    paths = f"{rtc_path}/*/*{polarization[0]}*.tif"
    dbl_polar = True
print(paths)

<font face="Calibri" size="3"><b>Write a function to collect the product acquisition dates:</b></font>

In [ ]:
def get_dates(paths, process_type):
    dates = []
    pths = glob.glob(paths)
    for p in pths:
        if process_type == 31:
            date = p.split("/")[-1].split("_")[4].split("T")[0]
        else:
            date = p.split("/")[-1].split("_")[3].split("T")[0]
        dates.append(date)
    dates.sort()
    return dates

<font face="Calibri" size="3"><b>Call get_dates() to collect the product acquisition dates:</b></font>

In [ ]:
dates = get_dates(paths, process_type)
print(dates)

<hr>
<font face="Calibri" size="3"> You may notice duplicates in your acquisition dates. As HyP3 processes SAR data on a frame-by-frame basis, duplicates may occur if your area of interest is covered by two consecutive  image frames. In this case, two separate images are generated that need to be merged together before time series processing can commence. 
<br><br>
<b>Write functions to collect and print the paths of the tiffs:</b> </font>

In [ ]:
def get_tiff_paths(paths):
    tiff_paths = !ls $paths | sort -t_ -k5,5
    return tiff_paths

def print_tiff_paths(tiff_paths):
    print("Tiff paths:")
    for p in tiff_paths:
        print(f"{p}\n")

<font face="Calibri" size="3"><b>Collect and print the paths of the tiffs:</b></font>

In [ ]:
tiff_paths = get_tiff_paths(paths)
#print_tiff_paths(tiff_paths)

<hr>
<font face="Calibri" size="4"> <b>1.2 Fix multiple UTM Zone-related issues</b> <br>
<br>
<font face="Calibri" size="3">Fix multiple UTM Zone-related issues should they exist in your data set. If multiple UTM zones are found, the following code cells will identify the predominant UTM zone and reproject the rest into that zone. This step must be completed prior to merging frames or performing any analysis.</font>
<br><br>
<font face="Calibri" size="3"><b>Use gdal.Info to determine the UTM definition types and zones in each product:</b></font>

In [ ]:
coord_choice = select_parameter('Coord Systems:', ["UTM", "Lat/Long"])
coord_choice

In [ ]:
utm_zones = []
utm_types = []
print('Checking UTM Zones in the data stack ...\n')
for k in range(0, len(tiff_paths)):
    info = (gdal.Info(tiff_paths[k], options = ['-json']))
    info = (json.loads(info))['coordinateSystem']['wkt']
    zone = info.split('ID')[-1].split(',')[1][0:-2]
    utm_zones.append(zone)
    typ = info.split('ID')[-1].split('"')[1]
    utm_types.append(typ)
print(f"UTM Zones:\n {utm_zones}\n")
print(f"UTM Types:\n {utm_types}")

<font face="Calibri" size="3"><b>Identify the most commonly used UTM Zone in the data:</b></font>

In [ ]:
if coord_choice.value == 'UTM':
    utm_unique, counts = np.unique(utm_zones, return_counts=True)
    a = np.where(counts == np.max(counts))
    predominant_utm = utm_unique[a][0]
    print(f"Predominant UTM Zone: {predominant_utm}")
else:
    predominant_utm = '4326'

<font face="Calibri" size="3"><b>Reproject images with errant UTMs to the predominant UTM:</b></font>

In [ ]:
reproject_indicies = [i for i, j in enumerate(utm_zones) if j != predominant_utm] #makes list of indicies in utm_zones that need to be reprojected
print('--------------------------------------------')
print('Reprojecting %4.1f files' %(len(reproject_indicies)))
print('--------------------------------------------')
for k in reproject_indicies:
    temppath = tiff_paths[k].strip()
    _, product_name, tiff_name = temppath.split('/')
    if coord_choice.value == 'UTM':
        cmd = f"gdalwarp -overwrite rtc_products/{product_name}/{tiff_name} rtc_products/{product_name}/r{tiff_name} -s_srs {utm_types[k]}:{utm_zones[k]} -t_srs EPSG:{predominant_utm}"
    elif coord_choice.value == 'Lat/Long':
        cmd = f"gdalwarp -overwrite rtc_products/{product_name}/{tiff_name} rtc_products/{product_name}/r{tiff_name} -s_srs {utm_types[k]}:{utm_zones[k]} -t_srs EPSG:4326"
    #print(f"Calling the command: {cmd}")
    !{cmd}
    rm_command = f"rm {tiff_paths[k].strip()}"
    #print(f"Calling the command: {rm_command}")
    !{rm_command}

In [ ]:
tiff_paths = get_tiff_paths(paths)
#print_tiff_paths(tiff_paths)

<hr>
<font face="Calibri" size="4"> <b>1.3 Merge multiple frames from the same date.</b></font>
<br><br>
<font face="Calibri" size="3"><b>Create a list of relative paths to the tiffs in each selected polarization:</b></font>

In [ ]:
if dbl_polar:
    rel_paths = [f"{rtc_path}/*/*{polarization.split(' ')[0]}.tif", 
                 f"{rtc_path}/*/*{polarization.split(' ')[2]}.tif"]
else:
    rel_paths = [paths]
                 
print(rel_paths)                 
dates = get_dates(rel_paths[0], process_type)                 
print(dates)

<font face="Calibri" size="3"><b>Create a set containing each represented date:</b></font>

In [ ]:
unique_dates = set(dates)
print(unique_dates)

<font face="Calibri" size="3"><b>Determine which dates have multiple frames. Create a dictionary with each date as a key linked to a value set as an empty string:</b></font>

In [ ]:
dup_date_batches = [{}]
for date in unique_dates:
    count = 0
    for d in dates:
        if date == d:
            count +=1
    if count > 1:
        dup_date_batches[0].update({date : ""})
if dbl_polar:
    dup_date_batches.append(copy.deepcopy(dup_date_batches[0]))
print(dup_date_batches)

<font face="Calibri" size="3"><b>Update the key values in dup_paths with the string paths to all the tiffs for each date:</b></font>

In [ ]:
for i, rel_pth in enumerate(rel_paths):
    polar_paths = get_tiff_paths(rel_pth)
    for pth in polar_paths:
        date = pth.split('/')[2].split('_')[3][:8]
        if date in dup_date_batches[i]:
            dup_date_batches[i][date] = f"{dup_date_batches[i][date]} {pth}"

for d in dup_date_batches:
    for dd in d:
        print(d)
    print("\n\n")

<font face="Calibri" size="3"><b>Merge all the frames for each date.</b></font>

In [ ]:
for i, dup_dates in enumerate(dup_date_batches):
    for dup_date in dup_dates:
        output = f"{dup_dates[dup_date].split('/')[0]}/{dup_dates[dup_date].split('/')[1]}/new{dup_dates[dup_date].split('/')[2].split(' ')[0]}"
        gdal_command = f"gdal_merge.py -o {output} {dup_dates[dup_date]}"
        print(f"\n\nCalling the command: {gdal_command}\n")
        !{gdal_command}
        for pth in dup_dates[dup_date].split(' '):
            if pth and path_exists(pth):
                os.remove(pth)
                print(f"Deleting: {pth}")

<hr>
<font face="Calibri" size="3"> <b>Verify that all duplicate dates were resolved:</b> </font>

In [ ]:
for rel_pth in rel_paths:
    dates = get_dates(rel_pth, process_type)
    if len(dates) != len(set(dates)):
        print(f"Duplicate dates still present!")
    else:
        print(f"No duplicate dates are associated with {rel_pth}")

<font face="Calibri" size="3"><b>Update the paths of the tiffs:</b></font>

In [ ]:
tiff_paths = get_tiff_paths(paths)
#print_tiff_paths(tiff_paths) # uncomment to view paths

<hr>
<font face="Calibri">

<font size="5"> <b> 2. Create Subset and Stack Up Your Data </b> </font> 

<font size="3"> Now you are ready to work with your data. The next cells allow you to select an area of interest (AOI; via bounding-box corner coordinates) for your data analysis. Once selected, the AOI is being extracted and a data stack is formed.

<b>As a first step, retrieve the maximum extent coordinates for the image stack so we can zoom into the coverage area on the Bokeh map:</b>
</font> 
</font>

In [ ]:
lower_left = [30000000, 30000000]
upper_right = [-30000000, -30000000]
for p in tiff_paths:
    info = (gdal.Info(p, options = ['-json']))
    l_l = (json.loads(info))['cornerCoordinates']['lowerLeft']
    u_r = (json.loads(info))['cornerCoordinates']['upperRight']
    if l_l[0] < lower_left[0]:
        lower_left[0] = l_l[0]
    if l_l[1] < lower_left[1]:
        lower_left[1] = l_l[1]
    if u_r[0] > upper_right[0]:
        upper_right[0] = u_r[0]
    if u_r[1] > upper_right[1]:
        upper_right[1] = u_r[1]

<font face="Calibri" size="3"> <b>Convert the coordinates to EPSG:3857 (web-mercator), which is the projection used by Bokeh:</b> </font> 

In [ ]:
out_proj = Proj(init="epsg:3857") #web mercator
in_proj = Proj(init=f"epsg:{predominant_utm}")
lower_left[0], lower_left[1] = transform(in_proj, out_proj, lower_left[0], lower_left[1])
upper_right[0], upper_right[1] = transform(in_proj, out_proj, upper_right[0], upper_right[1])
print(f"Lower Left Coord: {lower_left}")
print(f"Upper Right Coord: {upper_right}")

<font face="Calibri" size="3"><b>Create and display an interactive Area-of-Interest selector:</b></font> 

In [ ]:
display(Markdown(f"<text style=color:blue><b>Area of Interest Selector Tips:\n</b></text>"))
display(Markdown(f'<text style=color:blue>- The black bounding box shows a data stack\'s maximum extents</text>'))
display(Markdown(f'<text style=color:blue>- If revisiting this code cell out of sequence, rerun it in order to alter the current selection.</text>'))
display(Markdown(f'<text style=color:blue>- Select tools from the side menu.</text>'))
display(Markdown(f'<text style=color:blue>- Choose "Wheel Zoom" to use zoom with your mouse wheel.</text>'))
display(Markdown(f'<text style=color:blue>- Choose <b>"Box Select"</b> to select an area of interest.</text>'))
display(Markdown(f'<text style=color:blue>- Hitting "reset" resets the map display but does <b>not</b> alter your selection.</text>'))

aoi = AOI(lower_left, upper_right)
aoi.display_AOI()

<font face="Calibri" size="3"> <b>Convert the EPSG:3857 coords back to the predominant EPSG in the data stack:</b></font> 

In [ ]:
if not aoi.subset_coords[0][0]:
    print(f"WARNING: You must select a subset area of interest in the previous cell before continuing.")
    print(f"\nPlease make a selection and rerun this code cell.")
else:
    in_proj = Proj(init="epsg:3857")
    out_proj = Proj(init=f"epsg:{predominant_utm}")
    coords = [[None, None], [None, None]]
    coords[0][0], coords[0][1] = transform(in_proj, out_proj, 
                                           aoi.subset_coords[0][0], 
                                           aoi.subset_coords[1][1])
    coords[1][0], coords[1][1] = transform(in_proj, out_proj, 
                                           aoi.subset_coords[1][0], 
                                           aoi.subset_coords[0][1])
    print(coords)

<font size="3"> <b>Update the list of all the absolute paths of the tiffs:</b> </font> 

In [ ]:
tiff_paths = get_tiff_paths(paths)
#print_tiff_paths(tiff_paths) # uncomment to view paths

<font size="3"><b>Subset the tiffs and move them from the individual product directories into their own directory, /tiffs:</b></font> 

In [ ]:
!mkdir -p tiffs
if path_exists('tiffs'):
    product_number = 1
    for tiff_path in tiff_paths:
        _, granule_name, tiff_name = tiff_path.split('/')
        if process_type == 31:
            date = tiff_name.split('_')[4]
        else:
            date = tiff_name.split('_')[3]
        polar = tiff_name.split('.')[0][-2:]
        print(f"\nProduct #{product_number}:")
        product_number += 1
        gdal_command = f"gdal_translate -projwin {coords[0][0]} {coords[0][1]} {coords[1][0]} {coords[1][1]} -projwin_srs 'EPSG:{predominant_utm}' -co \"COMPRESS=DEFLATE\" -a_nodata 0 {tiff_path} tiffs/{date}_{polar}.tiff"
        print(f"Calling the command: {gdal_command}")
        !{gdal_command}

<font size="3"><b>Grab the updated paths of the images:</b></font>

In [ ]:
tp = f"{analysis_directory}/tiffs"
tiff_paths = get_tiff_paths(tp)
#print_tiff_paths(tiff_paths)

<font size="3"><b>Delete any subset tifs that are filled with NaNs and contain no data.</b></font>

In [ ]:
t_path = f"{os.getcwd()}/tiffs/"
tiff_paths = get_tiff_paths(t_path)
remove_nan_filled_tifs(t_path, tiff_paths)
print(f"\nThere are {len(tiff_paths)} tiffs remaining in the image stack.")

<font size="3"><b>Update the list of dates and tiff_paths after removing NaN filled images:</b></font>

In [ ]:
dates = []
pth = glob.glob(f"{t_path}/*.tiff")
pth.sort()
for p in pth:
    date = p[len(p)-23:len(p)-15]
    dates.append(date)
    print(date)

tiff_paths = get_tiff_paths(tp)
# print_tiff_paths(tiff_paths) # uncomment to print tiff paths

<hr>
<font face="Calibri" size="3"> Now we stack up the data by creating a virtual raster table with links to all subset data files: </font>
<br><br>
<font size="3"><b>Create the virtual raster table for the subset GeoTiffs:</b></font>

In [ ]:
!gdalbuildvrt -separate raster_stack.vrt tiffs/*.tiff

<hr>
<font face="Calibri">

<font size="5"> <b> 3. Now You Can Work With Your Data </b> </font> 

<font size="3"> Now you are ready to perform time series analysis on your data stack
</font> 
</font>

<br>
<font face="Calibri" size="4"> <b> 3.1 Define Data Directory and Path to VRT </b> </font> 
<br><br>
<font face="Calibri" size="3"><b>Create a variable containing the VRT filename:</b></font>

In [ ]:
image_file = "raster_stack.vrt"

<font face="Calibri" size="3"><b>Create an index of timedelta64 data with Pandas:</b></font>

In [ ]:
time_index = pd.DatetimeIndex(dates)

<font face="Calibri" size="3"><b>Print the bands and dates for all images in the virtual raster table (VRT):</b></font>

In [ ]:
j = 1
print(f"Bands and dates for {image_file}")
for i in time_index:
    print("{:4d} {}".format(j, i.date()), end=' ')
    j += 1
    if j%5 == 1: 
        print()

<hr>
<br>
<font face="Calibri" size="4"> <b> 3.2 Open Your Data Stack and Visualize Some Layers </b> </font> 

<font face="Calibri" size="3"> We will <b>open your VRT</b> and visualize some layers using Matplotlib. </font>

In [ ]:
img = gdal.Open(image_file)

<font face="Calibri" size="3"><b>Print the bands, pixels, and lines:</b></font>

In [ ]:
print(f"Number of  bands: {img.RasterCount}")
print(f"Number of pixels: {img.RasterXSize}")
print(f"Number of  lines: {img.RasterYSize}")

<font face="Calibri" size="3"><b>Read in raster data for the first two bands:</b></font>

In [ ]:
raster_1 = img.GetRasterBand(1).ReadAsArray() # change the number passed to GetRasterBand() to 
where_are_NaNs = np.isnan(raster_1)           # read rasters from different bands
raster_1[where_are_NaNs] = 0

raster_2 = img.GetRasterBand(2).ReadAsArray() #must pass a valid band number to GetRasterBand()
where_are_NaNs = np.isnan(raster_2)
raster_2[where_are_NaNs] = 0

<font face="Calibri" size="3"><b>Plot images and histograms for bands 1 and 2:</b>
<br>
Note: Depending the histograms plotted by this cell, you may wish to adjust vmax when calling imshow() on ax1 and ax3. Increase the vmax value if the histogram cuts off much of the end of the peak, making your image too bright to see features well. Decrease vmax if the histogram extends much beyond the end of the peak, which will make your image appear dark. </font>

In [ ]:
# Setup the pyplot plots
fig = plb.figure(figsize=(18,10)) # Initialize figure with a size
ax1 = fig.add_subplot(221)  # 221 determines: 2 rows, 2 plots, first plot
ax2 = fig.add_subplot(222)  # 222 determines: 2 rows, 2 plots, second plot
ax3 = fig.add_subplot(223)  # 223 determines: 2 rows, 2 plots, third plot
ax4 = fig.add_subplot(224)  # 224 determines: 2 rows, 2 plots, fourth plot

# Plot the band 1 image
band_number = 1
ax1.imshow(raster_1,cmap='gray', vmin=0, vmax=0.3) # see note above regarding vmax adjustments
ax1.set_title('Image Band {} {}'.format(band_number, time_index[band_number-1].date()))

# Flatten the band 1 image into a 1 dimensional vector and plot the histogram:
h = ax2.hist(raster_1.flatten(), bins=200, range=(0, 0.3))
ax2.xaxis.set_label_text('Amplitude? (Uncalibrated DN Values)')
ax2.set_title('Histogram Band {} {}'.format(band_number, time_index[band_number-1].date()))

# Plot the band 2 image
band_number = 2
ax3.imshow(raster_2,cmap='gray', vmin=0, vmax=0.3) # see note above regarding vmax adjustments
ax3.set_title('Image Band {} {}'.format(band_number, time_index[band_number-1].date()))

# Flatten the band 2 image into a 1 dimensional vector and plot the histogram:
h = ax4.hist(raster_2.flatten(),bins=200,range=(0,0.3))
ax4.xaxis.set_label_text('Amplitude? (Uncalibrated DN Values)')
ax4.set_title('Histogram Band {} {}'.format(band_number, time_index[band_number-1].date()))

<hr>
<br>
<font face="Calibri" size="4"> <b> 3.3 Calibration and Data Conversion between dB and Power Scales </b> </font>

<font face="Calibri" size="3"> <font color='rgba(200,0,0,0.2)'> <b>Note, that if your data were generated by HyP3, this step is not necessary!</b> HyP3 performs the full data calibration and provides you with calibrated data in power scale. </font>
    
If, your data is from a different source, however, calibration may be necessary to ensure that image gray values correspond to proper radar cross section information. 

Calibration coefficients for SAR data are often defined in the decibel (dB) scale due to the high dynamic range of the imaging system. For the L-band ALOS PALSAR data at hand, the conversion from uncalibrated DN values to calibrated radar cross section values in dB scale is performed by applying a standard **calibration factor of -83 dB**. 
<br> <br>
$\gamma^0_{dB} = 20 \cdot log10(DN) -83$

The data at hand are radiometrically terrain corrected images, which are often expressed as terrain flattened $\gamma^0$ backscattering coefficients. For forest and land cover monitoring applications $\gamma^o$ is the preferred metric.

<b>To apply the calibration constant for your data and export in *dB* scale, uncomment the following code cell</b>: </font> 

In [ ]:
 #caldB=20*np.log10(rasterstack)-83

<font face="Calibri" size="3"> While **dB**-scaled images are often "visually pleasing", they are often not a good basis for mathematical operations on data. For instance, when we compute the mean of observations, it makes a difference whether we do that in power or dB scale. Since dB scale is a logarithmic scale, we cannot simply average data in that scale. 
    
Please note that the **correct scale** in which operations need to be performed **is the power scale.** This is critical, e.g. when speckle filters are applied, spatial operations like block averaging are performed, or time series are analyzed.

To **convert from dB to power**, apply: $\gamma^o_{pwr} = 10^{\frac{\gamma^o_{dB}}{10}}$ </font>

In [ ]:
#calPwr=np.power(10.,caldB/10.)

<hr>
<br>
<font face="Calibri" size="4"> <b> 3.4 Create a Time Series Animation </b> </font>

<font face="Calibri" size="3">Now we are ready to create a time series animation from the calibrated SAR data.
<br><br>
<b>Create a directory in which to store our plots and animations:</b>
</font> 

In [ ]:
output_path = 'plots_animations'
new_directory(output_path)

<font face="Calibri" size="3"> Now we are ready to <b>create a time series animation</b> from the calibrated SAR data. </font> 

In [ ]:
band = img.GetRasterBand(1)
raster0 = band.ReadAsArray()
band_number = 0 # Needed for updates
raster_stack = img.ReadAsArray()
img = None

<font face="Calibri" size="3"><b>Create a masked raster stack:</b></font> 

In [ ]:
raster_stack_masked = np.ma.masked_where(raster_stack==0, raster_stack)

<font face="Calibri" size="3"><b>Generate a matplotlib time-series animation:</b></font> 

In [ ]:
%%capture
fig = plt.figure(figsize=(14, 8))
ax = fig.add_subplot(111)
ax.axis('off')
vmin = np.percentile(raster_stack.flatten(), 5)
vmax = np.percentile(raster_stack.flatten(), 95)

r0dB = 20 * np.log10(raster0) - 83

im = ax.imshow(raster0, cmap='gray', vmin=vmin, vmax=vmax)
ax.set_title("{}".format(time_index[0].date()))

def animate(i):
    ax.set_title("{}".format(time_index[i].date()))
    im.set_data(raster_stack[i])

# Interval is given in milliseconds
ani = animation.FuncAnimation(fig, animate, frames=raster_stack.shape[0], interval=400)

<font face="Calibri" size="3"><b>Configure matplotlib's RC settings for the animation:</b></font> 

In [ ]:
rc('animation', embed_limit=40971520.0) 

<font face="Calibri" size="3"><b>Create a javascript animation of the time-series running inline in the notebook:</b></font> 

In [ ]:
HTML(ani.to_jshtml())

<font face="Calibri" size="3"><b>Delete the dummy png</b> that was saved to the current working directory while generating the javascript animation in the last code cell.</font> 

In [ ]:
try:
    os.remove('None0000000.png')
except FileNotFoundError:
    pass

<font face="Calibri" size="3"><b>Save the animation (animation.gif):</b> </font> 

In [ ]:
ani.save(f"{output_path}/animation.gif", writer='pillow', fps=2)

<br>
<hr>
<font face="Calibri" size="4"> <b> 3.5 Plot the Time Series of Means Calculated Across the Subset </b> </font>

<font face="Calibri" size="3"> To create the time series of means, we will go through the following steps:
1. Ensure that you use the data in **power scale** ($\gamma^o_{pwr}$) for your mean calculations.
2. compute means.
3. convert the resulting mean values into dB scale for visualization.
4. plot time series of means. </font> 
<br><br>
<font face="Calibri" size="3"> <b>Compute the means:</b> </font>

In [ ]:
rs_means_pwr = np.mean(raster_stack_masked, axis=(1, 2))

<font face="Calibri" size="3"><b>Convert resulting mean value time-series to dB scale for visualization:</b></font>

In [ ]:
rs_means_dB = 10.*np.log10(rs_means_pwr)

<font face="Calibri" size="3"><b>Plot and save the time series of means (RCSoverTime.png):</b></font>

In [ ]:
plt.rcParams.update({'font.size': 14})
fig = plt.figure(figsize=(16, 4))
ax1 = fig.subplots()
window_length = len(rs_means_pwr)-1
if window_length % 2 == 0:
    window_length -= 1
polyorder = math.ceil(window_length*0.1)
yhat = scipy.signal.savgol_filter(rs_means_pwr, window_length, polyorder) 
ax1.plot(time_index, yhat, color='red', marker='o', markerfacecolor='white', linewidth=3, markersize=6)
ax1.plot(time_index, rs_means_pwr, color='gray', linewidth=0.5)
plt.grid()
ax1.set_xlabel('Date')
ax1.set_ylabel('$\overline{\gamma^o}$ [power]')
plt.savefig(f'{output_path}/RCSoverTime.png', dpi=72, transparent='true')

<br>
<hr>
<font face="Calibri" size="4"> <b> 3.6 Calculate Coefficient of Variance </b> </font>

<font face="Calibri" size="3"> The coefficient of variance describes how much the $\sigma_{0}$ or $\gamma_{0}$ measurements in a pixel vary over time. Hence, the coefficient of variance can indicate different vegetation cover and soil moisture regimes in your area.</font> 
<br><br>
<font face="Calibri" size="3"><b>Write a function to convert our plots into GeoTiffs:</b></font> 

In [ ]:
def geotiff_from_plot(source_image, out_filename, extent, cmap=None, vmin=None, vmax=None, interpolation=None, dpi=300):
    assert "." not in out_filename, 'Error: Do not include the file extension in out_filename'
    assert type(extent) == list and len(extent) == 2 and len(extent[0]) == 2 and len(
        extent[1]) == 2, 'Error: extent must be a list in the form [[upper_left_x, upper_left_y], [lower_right_x, lower_right_y]]'
    
    plt.figure()
    plt.axis('off')
    plt.imshow(source_image, cmap=cmap, vmin=vmin, vmax=vmax, interpolation=interpolation)
    temp = f"{out_filename}_temp.png"
    plt.savefig(temp, dpi=dpi, transparent='true', bbox_inches='tight', pad_inches=0)

    cmd = f"gdal_translate -of Gtiff -a_ullr {extent[0][0]} {extent[0][1]} {extent[1][0]} {extent[1][1]} -a_srs EPSG:{predominant_utm} {temp} {out_filename}.tiff"
    !{cmd}
    try:
        os.remove(temp)
    except FileNotFoundError:
        pass

<font face="Calibri" size="3"><b>Plot the Coefficient of Variance Map and save it as a png (Coeffvar.png):</b> </font> 

In [ ]:
test = np.var(raster_stack,0)
mtest = np.mean(raster_stack[raster_stack.nonzero()],0)
coeffvar = test/(mtest+0.001)

plt.rcParams.update({'font.size': 14})
fig = plt.figure(figsize=(13, 10))
ax = fig.subplots()
ax.axis('off')
vmin = np.percentile(coeffvar.flatten(), 5)
vmax = np.percentile(coeffvar.flatten(), 95)
ax.set_title('Coefficient of Variance Map')

im = ax.imshow(coeffvar, cmap='jet', vmin=vmin, vmax=vmax)
fig.colorbar(im, ax=ax)
plt.savefig(f'{output_path}/Coeffvar.png', dpi=300, transparent='true')

<font face="Calibri" size="3"><b>Save the coefficient of variance map as a GeoTiff (Coeffvar.tiff):</b></font> 

In [ ]:
%%capture
geotiff_from_plot(coeffvar, f'{output_path}/Coeffvar', coords, cmap='jet', vmin=vmin, vmax=vmax)

<br>
<hr>
<font face="Calibri" size="4"> <b> 3.7 Threshold Coefficient of Variance Map </b> </font>

<font face="Calibri" size="3"> This is an example how to threshold the derived coefficient of variance map. This can be useful, e.g., to detect areas of active agriculture.</font> 
<br><br>

<font face="Calibri" size="3"><b>Plot and save the coefficient of variance histogram and CDF (thresh_coeff_var_histogram.png):</b></font> 

In [ ]:
plt.rcParams.update({'font.size': 14})
fig = plt.figure(figsize=(14, 6)) # Initialize figure with a size
ax1 = fig.add_subplot(121)  # 121 determines: 2 rows, 2 plots, first plot
ax2 = fig.add_subplot(122)
# Second plot: Histogram
# IMPORTANT: To get a histogram, we first need to *flatten* 
# the two-dimensional image into a one-dimensional vector.
h = ax1.hist(coeffvar.flatten(), bins=200, range=(0, 0.03))
ax1.xaxis.set_label_text('Coefficient of Variation')
ax1.set_title('Coeffvar Histogram')
plt.grid()
n, bins, patches = ax2.hist(coeffvar.flatten(), bins=200, range=(0, 0.03), cumulative='True', density='True', histtype='step', label='Empirical')
ax2.xaxis.set_label_text('Coefficient of Variation')
ax2.set_title('Coeffvar CDF')
plt.grid()
plt.savefig(f'{output_path}/thresh_coeff_var_histogram.png', dpi=72, transparent='true')

<font face="Calibri" size="3"><b>Plot the Threshold Coefficient of Variance Map and save it as a png (Coeffvarthresh.png):</b> </font> 

In [ ]:
plt.rcParams.update({'font.size': 14})
outind = np.where(n > 0.80)
threshind = np.min(outind)
thresh = bins[threshind]
coeffvarthresh = np.copy(coeffvar)
coeffvarthresh[coeffvarthresh < thresh] = 0
coeffvarthresh[coeffvarthresh > 0.1] = 0
fig = plt.figure(figsize=(13, 10))
ax = fig.subplots()
ax.axis('off')
vmin = np.percentile(coeffvar.flatten(), 5)
vmax = np.percentile(coeffvar.flatten(), 95)
ax.set_title(r'Thresholded Coeffvar Map [$\alpha=95%$]')
im = ax.imshow(coeffvarthresh, cmap='jet', vmin=vmin, vmax=vmax)
bar = fig.colorbar(im, ax=ax)
plt.savefig(f'{output_path}/Coeffvarthresh.png', dpi=300, transparent='true')

<font face="Calibri" size="3"><b>Save the Threshold Coefficient of Variance Map as a GeoTiff (Coeffvarthresh.tiff):</b> </font> 

In [ ]:
%%capture
geotiff_from_plot(coeffvarthresh, f'{output_path}/Coeffvarthresh', coords, cmap='jet', vmin=vmin, vmax=vmax)

<font face="Calibri" size="2"> <i>GEOS 657 Microwave Remote Sensing - Version 1.0 - March 2019 </i>
</font>